In [356]:
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from functools import reduce
from collections import Counter
import numpy as np
import pickle as pkl
import preprocess_funcs
from preprocess_funcs import run_preprocess, text_edit, do_replacements
import time

In [357]:
import importlib
importlib.reload(preprocess_funcs)

<module 'preprocess_funcs' from '/Users/bogoksel/duplicate-info-detection/preprocess_funcs.py'>

In [348]:
data_path = "data/df_preprocessed.pkl"

In [477]:
df_main = pd.read_pickle(data_path)

In [478]:
df_main.columns

Index(['İl', 'İlçe', 'Mahalle', 'Adres', 'Bulvar/Cadde/Sokak/Yol/Yanyol',
       'Bina Adı', 'Dış Kapı/ Blok/Apartman No', 'Kat', 'İç Kapı', 'Ad-Soyad',
       'Kaynak', 'Telefon', 'Oluşturulma Tarihi', 'Güncellenme Tarihi', 'id',
       'group', 'new_adres'],
      dtype='object')

In [350]:
df_main["merged_address"] = df_main['Bina Adı'] + " " + df_main['Dış Kapı/ Blok/Apartman No'] \
    + " " + df_main["Bulvar/Cadde/Sokak/Yol/Yanyol"] + " " + df_main["new_adres"]

In [351]:
grouped = df_main.groupby(['İl', 'İlçe', 'Mahalle'])

In [363]:
groups = list(grouped.__iter__())
example_group = groups[12][1]

In [364]:
from sklearn.cluster import DBSCAN

In [371]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(example_group["merged_address"])
dbscan = DBSCAN(eps=0.1, min_samples=2, metric="cosine").fit(vectors)

In [372]:
dbscan.labels_

array([-1,  0, -1,  0, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1])

In [382]:
dbscan.get_params()

{'algorithm': 'auto',
 'eps': 0.1,
 'leaf_size': 30,
 'metric': 'cosine',
 'metric_params': None,
 'min_samples': 2,
 'n_jobs': None,
 'p': None}

In [352]:
def add_similarity_columns_for_key(df, key_column_name):
    all_keys = df[key_column_name]
    all_ids = df["id"].values
    vectorizer = TfidfVectorizer(ngram_range=(1, 1))
    tfidf_vectors = vectorizer.fit_transform(all_keys)
    similarity_matrix = (tfidf_vectors * tfidf_vectors.T).toarray()
    second_most_similar_idx_per_row = similarity_matrix.argpartition(-2, axis=1)[:, -2].flatten()
    second_most_similar_similarity_per_row = np.take_along_axis(similarity_matrix, second_most_similar_idx_per_row[:, None], axis=1).flatten()
    second_most_similar_id_per_row = np.take(all_ids, second_most_similar_idx_per_row).flatten()
    df[f'similar_{key_column_name}_row_id'] = second_most_similar_id_per_row
    df[f'similar_{key_column_name}_row_similarity'] = second_most_similar_similarity_per_row
    return df

In [359]:
def get_new_rows():
    new_rows = []
    for _, group_df in grouped:
        try:
            if len(group_df) < 2:
                # Skip groups with less than 2 rows since you cannot compute the second most similar row
                continue
            preprocess_funcs.do_replacements(group_df, col="merged_address")
            with_similar_name_columns = add_similarity_columns_for_key(group_df, "Ad-Soyad")
            with_similar_name_columns_only = with_similar_name_columns[with_similar_name_columns["similar_Ad-Soyad_row_similarity"] > 0.5]
            with_similar_address_columns = add_similarity_columns_for_key(with_similar_name_columns_only, "merged_address")
            if len(with_similar_address_columns) > 0:
                new_rows.append(with_similar_address_columns)
        except ValueError as e:
            #print(f"Error for group {group_info}: {e}")
            continue
    return new_rows

In [360]:
rows = get_new_rows()

/var/folders/74/dsz72cwx4bq1dn01kwk2q8_80000gn/T/ipykernel_98380/4125724932.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'similar_{key_column_name}_row_id'] = second_most_similar_id_per_row
/var/folders/74/dsz72cwx4bq1dn01kwk2q8_80000gn/T/ipykernel_98380/4125724932.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'similar_{key_column_name}_row_similarity'] = second_most_similar_similarity_per_row
/var/folders/74/dsz72cwx4bq1dn01kwk2q8_80000gn/T/ipykernel_98380/4125724932.py:10: SettingWi

In [361]:
final_df = pd.concat(rows)
final_df.to_excel("deneme2.xlsx", index=False)

In [458]:
def cluster_by_column(
    df: pd.DataFrame,
    key_column_name: str,
    cluster_column_name: str,
    similarity_threshold: float = 0.1,
    tfidf_ngram_range: tuple = (1, 1),
    df_mask = None,
) -> pd.DataFrame:
    name_vectorizer = TfidfVectorizer(ngram_range=tfidf_ngram_range)
    name_tfidf_vectors = name_vectorizer.fit_transform(df.loc[df_mask, key_column_name])
    name_dbscan = DBSCAN(eps=similarity_threshold, min_samples=2, metric="cosine").fit(name_tfidf_vectors)
    df.loc[df_mask, cluster_column_name] = name_dbscan.labels_
    return df

def cluster_data(
    df: pd.DataFrame,
    name_similarity_threshold: float = 0.1,
    address_similarity_threshold: float = 0.1,
    tfidf_ngram_range: tuple = (1, 1),
) -> pd.DataFrame:
    def cluster_group(group_df):
        try:
            trivial_mask = (group_df["Ad-Soyad"] != "")
            group_df = cluster_by_column(group_df, "Ad-Soyad", "Ad-Soyad-cluster", name_similarity_threshold, tfidf_ngram_range, df_mask = trivial_mask)
            group_df.loc[group_df["Ad-Soyad"] == "", "Ad-Soyad-cluster"] = -1
        except ValueError as e:
            group_df["Ad-Soyad-cluster"] = -1
        name_clusters = group_df["Ad-Soyad-cluster"].unique()
        for name_cluster in name_clusters:
            cluster_df_mask = (group_df["Ad-Soyad-cluster"] == name_cluster)
            if name_cluster == -1:
                group_df.loc[cluster_df_mask, 'merged_address-cluster'] = -1
                continue
            try:
                group_df = cluster_by_column(group_df, "merged_address", "merged_address-cluster", address_similarity_threshold, tfidf_ngram_range, df_mask=cluster_df_mask)
            except ValueError as e:
                group_df.loc[cluster_df_mask, 'merged_address-cluster'] = -1
        return group_df
    return df.groupby(["İl", "İlçe", "Mahalle"]).apply(cluster_group)

In [460]:
grouped = cluster_data(df_main)

/var/folders/74/dsz72cwx4bq1dn01kwk2q8_80000gn/T/ipykernel_98380/2803250382.py:39: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby(["İl", "İlçe", "Mahalle"]).apply(cluster_group)


In [461]:
grouped.columns

Index(['İl', 'İlçe', 'Mahalle', 'Adres', 'Bulvar/Cadde/Sokak/Yol/Yanyol',
       'Bina Adı', 'Dış Kapı/ Blok/Apartman No', 'Kat', 'İç Kapı', 'Ad-Soyad',
       'Kaynak', 'Telefon', 'Oluşturulma Tarihi', 'Güncellenme Tarihi', 'id',
       'group', 'new_adres', 'merged_address', 'Ad-Soyad-cluster',
       'merged_address-cluster'],
      dtype='object')

In [474]:
duplicate_cluster_idx = (grouped["Ad-Soyad-cluster"] != -1) & (grouped["merged_address-cluster"] != -1)
grouped_duplicates = grouped[duplicate_cluster_idx]
c_grouped = grouped_duplicates.groupby(["İl", "İlçe", "Mahalle", "Ad-Soyad-cluster", "merged_address-cluster"])

In [483]:
list(c_grouped.__iter__())[5][1]

,İl,İlçe,Mahalle,Adres,Bulvar/Cadde/Sokak/Yol/Yanyol,Bina Adı,Dış Kapı/ Blok/Apartman No,Kat,İç Kapı,Ad-Soyad,Kaynak,Telefon,Oluşturulma Tarihi,Güncellenme Tarihi,id,group,new_adres,merged_address,Ad-Soyad-cluster,merged_address-cluster
12912,adana,cukurova,guzelyali,81189 sok no 10 25 mete aprt cukurova adana,81189. Sokak,Mete Apartmanı,No: 10,,25,arkadasimin anneannesi kardesi ve yegeni de on...,,,,,12912.0,ADANA_ÇUKUROVA_GÜZELYALI Mahallesi_Mete Apartm...,81189 sokak no 10 25 mete aprt,Mete Apartmanı No: 10 81189. Sokak 81189 sokak...,5.0,0.0
20403,adana,cukurova,guzelyali,81189 sok no 10 25 mete aprt cukurova adana,81189. Sokak,Mete Apartmanı,,,,arkadasimin anneannesi kardesi ve yegeni de on...,https://twitter.com/OgulcanBaran08/status/1622...,,,,20403.0,ADANA_ÇUKUROVA_GÜZELYALI Mahallesi_Mete Apartm...,81189 sokak no 10 25 mete aprt,Mete Apartmanı 81189. Sokak 81189 sokak no 10...,5.0,0.0
